Sensitivity Analysis
====================

### [Neil D. Lawrence](http://inverseprobability.com)

### 2020-11-05

**Abstract**: This week we introduce sensitivity analysis through
Emukit, showing how Emukit can deliver Sobol indices for understanding
how the output of the system is affected by different inputs.

$$
\newcommand{\tk}[1]{}
\newcommand{\Amatrix}{\mathbf{A}}
\newcommand{\KL}[2]{\text{KL}\left( #1\,\|\,#2 \right)}
\newcommand{\Kaast}{\kernelMatrix_{\mathbf{ \ast}\mathbf{ \ast}}}
\newcommand{\Kastu}{\kernelMatrix_{\mathbf{ \ast} \inducingVector}}
\newcommand{\Kff}{\kernelMatrix_{\mappingFunctionVector \mappingFunctionVector}}
\newcommand{\Kfu}{\kernelMatrix_{\mappingFunctionVector \inducingVector}}
\newcommand{\Kuast}{\kernelMatrix_{\inducingVector \bf\ast}}
\newcommand{\Kuf}{\kernelMatrix_{\inducingVector \mappingFunctionVector}}
\newcommand{\Kuu}{\kernelMatrix_{\inducingVector \inducingVector}}
\newcommand{\Kuui}{\Kuu^{-1}}
\newcommand{\Qaast}{\mathbf{Q}_{\bf \ast \ast}}
\newcommand{\Qastf}{\mathbf{Q}_{\ast \mappingFunction}}
\newcommand{\Qfast}{\mathbf{Q}_{\mappingFunctionVector \bf \ast}}
\newcommand{\Qff}{\mathbf{Q}_{\mappingFunctionVector \mappingFunctionVector}}
\newcommand{\aMatrix}{\mathbf{A}}
\newcommand{\aScalar}{a}
\newcommand{\aVector}{\mathbf{a}}
\newcommand{\acceleration}{a}
\newcommand{\bMatrix}{\mathbf{B}}
\newcommand{\bScalar}{b}
\newcommand{\bVector}{\mathbf{b}}
\newcommand{\basisFunc}{\phi}
\newcommand{\basisFuncVector}{\boldsymbol{ \basisFunc}}
\newcommand{\basisFunction}{\phi}
\newcommand{\basisLocation}{\mu}
\newcommand{\basisMatrix}{\boldsymbol{ \Phi}}
\newcommand{\basisScalar}{\basisFunction}
\newcommand{\basisVector}{\boldsymbol{ \basisFunction}}
\newcommand{\activationFunction}{\phi}
\newcommand{\activationMatrix}{\boldsymbol{ \Phi}}
\newcommand{\activationScalar}{\basisFunction}
\newcommand{\activationVector}{\boldsymbol{ \basisFunction}}
\newcommand{\bigO}{\mathcal{O}}
\newcommand{\binomProb}{\pi}
\newcommand{\cMatrix}{\mathbf{C}}
\newcommand{\cbasisMatrix}{\hat{\boldsymbol{ \Phi}}}
\newcommand{\cdataMatrix}{\hat{\dataMatrix}}
\newcommand{\cdataScalar}{\hat{\dataScalar}}
\newcommand{\cdataVector}{\hat{\dataVector}}
\newcommand{\centeredKernelMatrix}{\mathbf{ \MakeUppercase{\centeredKernelScalar}}}
\newcommand{\centeredKernelScalar}{b}
\newcommand{\centeredKernelVector}{\centeredKernelScalar}
\newcommand{\centeringMatrix}{\mathbf{H}}
\newcommand{\chiSquaredDist}[2]{\chi_{#1}^{2}\left(#2\right)}
\newcommand{\chiSquaredSamp}[1]{\chi_{#1}^{2}}
\newcommand{\conditionalCovariance}{\boldsymbol{ \Sigma}}
\newcommand{\coregionalizationMatrix}{\mathbf{B}}
\newcommand{\coregionalizationScalar}{b}
\newcommand{\coregionalizationVector}{\mathbf{ \coregionalizationScalar}}
\newcommand{\covDist}[2]{\text{cov}_{#2}\left(#1\right)}
\newcommand{\covSamp}[1]{\text{cov}\left(#1\right)}
\newcommand{\covarianceScalar}{c}
\newcommand{\covarianceVector}{\mathbf{ \covarianceScalar}}
\newcommand{\covarianceMatrix}{\mathbf{C}}
\newcommand{\covarianceMatrixTwo}{\boldsymbol{ \Sigma}}
\newcommand{\croupierScalar}{s}
\newcommand{\croupierVector}{\mathbf{ \croupierScalar}}
\newcommand{\croupierMatrix}{\mathbf{ \MakeUppercase{\croupierScalar}}}
\newcommand{\dataDim}{p}
\newcommand{\dataIndex}{i}
\newcommand{\dataIndexTwo}{j}
\newcommand{\dataMatrix}{\mathbf{Y}}
\newcommand{\dataScalar}{y}
\newcommand{\dataSet}{\mathcal{D}}
\newcommand{\dataStd}{\sigma}
\newcommand{\dataVector}{\mathbf{ \dataScalar}}
\newcommand{\decayRate}{d}
\newcommand{\degreeMatrix}{\mathbf{ \MakeUppercase{\degreeScalar}}}
\newcommand{\degreeScalar}{d}
\newcommand{\degreeVector}{\mathbf{ \degreeScalar}}
\newcommand{\diag}[1]{\text{diag}\left(#1\right)}
\newcommand{\diagonalMatrix}{\mathbf{D}}
\newcommand{\diff}[2]{\frac{\text{d}#1}{\text{d}#2}}
\newcommand{\diffTwo}[2]{\frac{\text{d}^2#1}{\text{d}#2^2}}
\newcommand{\displacement}{x}
\newcommand{\displacementVector}{\textbf{\displacement}}
\newcommand{\distanceMatrix}{\mathbf{ \MakeUppercase{\distanceScalar}}}
\newcommand{\distanceScalar}{d}
\newcommand{\distanceVector}{\mathbf{ \distanceScalar}}
\newcommand{\eigenvaltwo}{\ell}
\newcommand{\eigenvaltwoMatrix}{\mathbf{L}}
\newcommand{\eigenvaltwoVector}{\mathbf{l}}
\newcommand{\eigenvalue}{\lambda}
\newcommand{\eigenvalueMatrix}{\boldsymbol{ \Lambda}}
\newcommand{\eigenvalueVector}{\boldsymbol{ \lambda}}
\newcommand{\eigenvector}{\mathbf{ \eigenvectorScalar}}
\newcommand{\eigenvectorMatrix}{\mathbf{U}}
\newcommand{\eigenvectorScalar}{u}
\newcommand{\eigenvectwo}{\mathbf{v}}
\newcommand{\eigenvectwoMatrix}{\mathbf{V}}
\newcommand{\eigenvectwoScalar}{v}
\newcommand{\entropy}[1]{\mathcal{H}\left(#1\right)}
\newcommand{\errorFunction}{E}
\newcommand{\expDist}[2]{\left<#1\right>_{#2}}
\newcommand{\expSamp}[1]{\left<#1\right>}
\newcommand{\expectation}[1]{\left\langle #1 \right\rangle }
\newcommand{\expectationDist}[2]{\left\langle #1 \right\rangle _{#2}}
\newcommand{\expectedDistanceMatrix}{\mathcal{D}}
\newcommand{\eye}{\mathbf{I}}
\newcommand{\fantasyDim}{r}
\newcommand{\fantasyMatrix}{\mathbf{ \MakeUppercase{\fantasyScalar}}}
\newcommand{\fantasyScalar}{z}
\newcommand{\fantasyVector}{\mathbf{ \fantasyScalar}}
\newcommand{\featureStd}{\varsigma}
\newcommand{\gammaCdf}[3]{\mathcal{GAMMA CDF}\left(#1|#2,#3\right)}
\newcommand{\gammaDist}[3]{\mathcal{G}\left(#1|#2,#3\right)}
\newcommand{\gammaSamp}[2]{\mathcal{G}\left(#1,#2\right)}
\newcommand{\gaussianDist}[3]{\mathcal{N}\left(#1|#2,#3\right)}
\newcommand{\gaussianSamp}[2]{\mathcal{N}\left(#1,#2\right)}
\newcommand{\given}{|}
\newcommand{\half}{\frac{1}{2}}
\newcommand{\heaviside}{H}
\newcommand{\hiddenMatrix}{\mathbf{ \MakeUppercase{\hiddenScalar}}}
\newcommand{\hiddenScalar}{h}
\newcommand{\hiddenVector}{\mathbf{ \hiddenScalar}}
\newcommand{\identityMatrix}{\eye}
\newcommand{\inducingInputScalar}{z}
\newcommand{\inducingInputVector}{\mathbf{ \inducingInputScalar}}
\newcommand{\inducingInputMatrix}{\mathbf{Z}}
\newcommand{\inducingScalar}{u}
\newcommand{\inducingVector}{\mathbf{ \inducingScalar}}
\newcommand{\inducingMatrix}{\mathbf{U}}
\newcommand{\inlineDiff}[2]{\text{d}#1/\text{d}#2}
\newcommand{\inputDim}{q}
\newcommand{\inputMatrix}{\mathbf{X}}
\newcommand{\inputScalar}{x}
\newcommand{\inputSpace}{\mathcal{X}}
\newcommand{\inputVals}{\inputVector}
\newcommand{\inputVector}{\mathbf{ \inputScalar}}
\newcommand{\iterNum}{k}
\newcommand{\kernel}{\kernelScalar}
\newcommand{\kernelMatrix}{\mathbf{K}}
\newcommand{\kernelScalar}{k}
\newcommand{\kernelVector}{\mathbf{ \kernelScalar}}
\newcommand{\kff}{\kernelScalar_{\mappingFunction \mappingFunction}}
\newcommand{\kfu}{\kernelVector_{\mappingFunction \inducingScalar}}
\newcommand{\kuf}{\kernelVector_{\inducingScalar \mappingFunction}}
\newcommand{\kuu}{\kernelVector_{\inducingScalar \inducingScalar}}
\newcommand{\lagrangeMultiplier}{\lambda}
\newcommand{\lagrangeMultiplierMatrix}{\boldsymbol{ \Lambda}}
\newcommand{\lagrangian}{L}
\newcommand{\laplacianFactor}{\mathbf{ \MakeUppercase{\laplacianFactorScalar}}}
\newcommand{\laplacianFactorScalar}{m}
\newcommand{\laplacianFactorVector}{\mathbf{ \laplacianFactorScalar}}
\newcommand{\laplacianMatrix}{\mathbf{L}}
\newcommand{\laplacianScalar}{\ell}
\newcommand{\laplacianVector}{\mathbf{ \ell}}
\newcommand{\latentDim}{q}
\newcommand{\latentDistanceMatrix}{\boldsymbol{ \Delta}}
\newcommand{\latentDistanceScalar}{\delta}
\newcommand{\latentDistanceVector}{\boldsymbol{ \delta}}
\newcommand{\latentForce}{f}
\newcommand{\latentFunction}{u}
\newcommand{\latentFunctionVector}{\mathbf{ \latentFunction}}
\newcommand{\latentFunctionMatrix}{\mathbf{ \MakeUppercase{\latentFunction}}}
\newcommand{\latentIndex}{j}
\newcommand{\latentScalar}{z}
\newcommand{\latentVector}{\mathbf{ \latentScalar}}
\newcommand{\latentMatrix}{\mathbf{Z}}
\newcommand{\learnRate}{\eta}
\newcommand{\lengthScale}{\ell}
\newcommand{\rbfWidth}{\ell}
\newcommand{\likelihoodBound}{\mathcal{L}}
\newcommand{\likelihoodFunction}{L}
\newcommand{\locationScalar}{\mu}
\newcommand{\locationVector}{\boldsymbol{ \locationScalar}}
\newcommand{\locationMatrix}{\mathbf{M}}
\newcommand{\variance}[1]{\text{var}\left( #1 \right)}
\newcommand{\mappingFunction}{f}
\newcommand{\mappingFunctionMatrix}{\mathbf{F}}
\newcommand{\mappingFunctionTwo}{g}
\newcommand{\mappingFunctionTwoMatrix}{\mathbf{G}}
\newcommand{\mappingFunctionTwoVector}{\mathbf{ \mappingFunctionTwo}}
\newcommand{\mappingFunctionVector}{\mathbf{ \mappingFunction}}
\newcommand{\scaleScalar}{s}
\newcommand{\mappingScalar}{w}
\newcommand{\mappingVector}{\mathbf{ \mappingScalar}}
\newcommand{\mappingMatrix}{\mathbf{W}}
\newcommand{\mappingScalarTwo}{v}
\newcommand{\mappingVectorTwo}{\mathbf{ \mappingScalarTwo}}
\newcommand{\mappingMatrixTwo}{\mathbf{V}}
\newcommand{\maxIters}{K}
\newcommand{\meanMatrix}{\mathbf{M}}
\newcommand{\meanScalar}{\mu}
\newcommand{\meanTwoMatrix}{\mathbf{M}}
\newcommand{\meanTwoScalar}{m}
\newcommand{\meanTwoVector}{\mathbf{ \meanTwoScalar}}
\newcommand{\meanVector}{\boldsymbol{ \meanScalar}}
\newcommand{\mrnaConcentration}{m}
\newcommand{\naturalFrequency}{\omega}
\newcommand{\neighborhood}[1]{\mathcal{N}\left( #1 \right)}
\newcommand{\neilurl}{http://inverseprobability.com/}
\newcommand{\noiseMatrix}{\boldsymbol{ E}}
\newcommand{\noiseScalar}{\epsilon}
\newcommand{\noiseVector}{\boldsymbol{ \epsilon}}
\newcommand{\norm}[1]{\left\Vert #1 \right\Vert}
\newcommand{\normalizedLaplacianMatrix}{\hat{\mathbf{L}}}
\newcommand{\normalizedLaplacianScalar}{\hat{\ell}}
\newcommand{\normalizedLaplacianVector}{\hat{\mathbf{ \ell}}}
\newcommand{\numActive}{m}
\newcommand{\numBasisFunc}{m}
\newcommand{\numComponents}{m}
\newcommand{\numComps}{K}
\newcommand{\numData}{n}
\newcommand{\numFeatures}{K}
\newcommand{\numHidden}{h}
\newcommand{\numInducing}{m}
\newcommand{\numLayers}{\ell}
\newcommand{\numNeighbors}{K}
\newcommand{\numSequences}{s}
\newcommand{\numSuccess}{s}
\newcommand{\numTasks}{m}
\newcommand{\numTime}{T}
\newcommand{\numTrials}{S}
\newcommand{\outputIndex}{j}
\newcommand{\paramVector}{\boldsymbol{ \theta}}
\newcommand{\parameterMatrix}{\boldsymbol{ \Theta}}
\newcommand{\parameterScalar}{\theta}
\newcommand{\parameterVector}{\boldsymbol{ \parameterScalar}}
\newcommand{\partDiff}[2]{\frac{\partial#1}{\partial#2}}
\newcommand{\precisionScalar}{j}
\newcommand{\precisionVector}{\mathbf{ \precisionScalar}}
\newcommand{\precisionMatrix}{\mathbf{J}}
\newcommand{\pseudotargetScalar}{\widetilde{y}}
\newcommand{\pseudotargetVector}{\mathbf{ \pseudotargetScalar}}
\newcommand{\pseudotargetMatrix}{\mathbf{ \widetilde{Y}}}
\newcommand{\rank}[1]{\text{rank}\left(#1\right)}
\newcommand{\rayleighDist}[2]{\mathcal{R}\left(#1|#2\right)}
\newcommand{\rayleighSamp}[1]{\mathcal{R}\left(#1\right)}
\newcommand{\responsibility}{r}
\newcommand{\rotationScalar}{r}
\newcommand{\rotationVector}{\mathbf{ \rotationScalar}}
\newcommand{\rotationMatrix}{\mathbf{R}}
\newcommand{\sampleCovScalar}{s}
\newcommand{\sampleCovVector}{\mathbf{ \sampleCovScalar}}
\newcommand{\sampleCovMatrix}{\mathbf{s}}
\newcommand{\scalarProduct}[2]{\left\langle{#1},{#2}\right\rangle}
\newcommand{\sign}[1]{\text{sign}\left(#1\right)}
\newcommand{\sigmoid}[1]{\sigma\left(#1\right)}
\newcommand{\singularvalue}{\ell}
\newcommand{\singularvalueMatrix}{\mathbf{L}}
\newcommand{\singularvalueVector}{\mathbf{l}}
\newcommand{\sorth}{\mathbf{u}}
\newcommand{\spar}{\lambda}
\newcommand{\trace}[1]{\text{tr}\left(#1\right)}
\newcommand{\BasalRate}{B}
\newcommand{\DampingCoefficient}{C}
\newcommand{\DecayRate}{D}
\newcommand{\Displacement}{X}
\newcommand{\LatentForce}{F}
\newcommand{\Mass}{M}
\newcommand{\Sensitivity}{S}
\newcommand{\basalRate}{b}
\newcommand{\dampingCoefficient}{c}
\newcommand{\mass}{m}
\newcommand{\sensitivity}{s}
\newcommand{\springScalar}{\kappa}
\newcommand{\springVector}{\boldsymbol{ \kappa}}
\newcommand{\springMatrix}{\boldsymbol{ \mathcal{K}}}
\newcommand{\tfConcentration}{p}
\newcommand{\tfDecayRate}{\delta}
\newcommand{\tfMrnaConcentration}{f}
\newcommand{\tfVector}{\mathbf{ \tfConcentration}}
\newcommand{\velocity}{v}
\newcommand{\sufficientStatsScalar}{g}
\newcommand{\sufficientStatsVector}{\mathbf{ \sufficientStatsScalar}}
\newcommand{\sufficientStatsMatrix}{\mathbf{G}}
\newcommand{\switchScalar}{s}
\newcommand{\switchVector}{\mathbf{ \switchScalar}}
\newcommand{\switchMatrix}{\mathbf{S}}
\newcommand{\tr}[1]{\text{tr}\left(#1\right)}
\newcommand{\loneNorm}[1]{\left\Vert #1 \right\Vert_1}
\newcommand{\ltwoNorm}[1]{\left\Vert #1 \right\Vert_2}
\newcommand{\onenorm}[1]{\left\vert#1\right\vert_1}
\newcommand{\twonorm}[1]{\left\Vert #1 \right\Vert}
\newcommand{\vScalar}{v}
\newcommand{\vVector}{\mathbf{v}}
\newcommand{\vMatrix}{\mathbf{V}}
\newcommand{\varianceDist}[2]{\text{var}_{#2}\left( #1 \right)}
\newcommand{\vecb}[1]{\left(#1\right):}
\newcommand{\weightScalar}{w}
\newcommand{\weightVector}{\mathbf{ \weightScalar}}
\newcommand{\weightMatrix}{\mathbf{W}}
\newcommand{\weightedAdjacencyMatrix}{\mathbf{A}}
\newcommand{\weightedAdjacencyScalar}{a}
\newcommand{\weightedAdjacencyVector}{\mathbf{ \weightedAdjacencyScalar}}
\newcommand{\onesVector}{\mathbf{1}}
\newcommand{\zerosVector}{\mathbf{0}}
$$

<!-- Do not edit this file locally. -->
<!-- Do not edit this file locally. -->
<!---->
<!-- Do not edit this file locally. -->
<!-- Do not edit this file locally. -->
<!-- The last names to be defined. Should be defined entirely in terms of macros from above-->
<!--

-->

In [ ]:
%pip install emukit

In [ ]:
%pip install gpy

Emukit Sensitivity Analysis
---------------------------

(Kennedy and O’Hagan, 2000; Saltelli et al., 2010, 2008, 2004; Sobol,
2001, 1990)

This introduction is based on [Introduction to Global Sensitivity
Analysis with
Emukit](https://github.com/EmuKit/emukit/blob/master/notebooks/Emukit-tutorial-sensitivity-montecarlo.ipynb)
written by Mark Pullin, Javier Gonzalez, Juan Emmanuel Johnson and
Andrei Paleyes.

> A possible definition of sensitivity analysis is the following: The
> study of how uncertainty in the output of a model (numerical or
> otherwise) can be apportioned to different sources of uncertainty in
> the model input (Saltelli et al., 2004). A related practice is
> ‘uncertainty analysis’, which focuses rather on quantifying
> uncertainty in model output. Ideally, uncertainty and sensitivity
> analyses should be run in tandem, with uncertainty analysis preceding
> in current practice.
>
> In Chapter 1 of Saltelli et al. (2008)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from matplotlib import colors as mcolors
from matplotlib import cm

In [ ]:
import urllib.request

In [ ]:
urllib.request.urlretrieve('https://raw.githubusercontent.com/lawrennd/talks/gh-pages/mlai.py','mlai.py')

In [ ]:
urllib.request.urlretrieve('https://raw.githubusercontent.com/lawrennd/talks/gh-pages/teaching_plots.py','teaching_plots.py')

In [ ]:
urllib.request.urlretrieve('https://raw.githubusercontent.com/lawrennd/talks/gh-pages/gp_tutorial.py','gp_tutorial.py')

In [ ]:
%pip install pyDOE

In [ ]:
import mlai
import teaching_plots as plot

Sensitivity analysis is a statistical technique widely used to test the
reliability of real systems. Imagine a simulator of taxis picking up
customers in a city like the one showed in the [Emukit
playground](https://github.com/amzn/emukit-playground). The profit of
the taxi company depends on factors like the number of taxis on the road
and the price per trip. In this example, a global sensitivity analysis
of the simulator could be useful to decompose the variance of the profit
in a way that can be assigned to the input variables of the simulator.

There are different ways of doing a sensitivity analysis of the
variables of a simulator. In this notebook we will start with an
approach based on Monte Carlo sampling that is useful when evaluating
the simulator is cheap. If evaluating the simulator is expensive,
emulators can then be used to speed up computations. We will show this
in the last part of the notebook. Next, we start with a few formal
definitions and literature review so we can understand the basics of
Sensitivity Analysis and how it can performed with Emukit.

Any simulator can be viewed as a function $$
y=f(\mathbf{ x}),
$$ where $\mathbf{ x}$ is a vector of $p$ uncertain model inputs
$x_1,\dots,x_p$, and $y$ is some univariate model output. We assume that
$f$ is a square integrable function and that the inputs are
statistically independent and uniformly distributed within the hypercube
$x_i \in [0,1]$ for $i=1,2,\dots,p$, although the bounds can be
generalized. The Sobol decomposition of $f(\cdot)$ allows us to write it
as $$
y= f_0 + \sum_{i=1}^pf_i(x_i) + \sum_{i<j}^{p} f_{ij}(x_i,x_j) + \cdots + f_{1,2,\dots,p}(x_1,x_2,\dots,x_p),
$$ where $f_0$ is a constant term, $f_i$ is a function of $x_i$,
$f_{ij}$ a function of $x_i$ and $x_j$, etc. A condition of this
decomposition is that, $$ 
\int_0^1 f_{i_1 i_2 \dots i_p}(x_{i_1},x_{i_2},\dots,x_{i_p}) \text{d}x_{k}=0, \text{ for } k = i_1,...,i_p. 
$$ This means that all the terms in the decomposition are orthogonal,
which can be written in terms of conditional expected values as
$$\begin{align*}
f_0 &= E(y) \\
f_i(x_i) & = E(y|x_i) - f_0 \\
f_{ij}(x_i,x_j) & = E(y|x_i,x_j) - f_0 - f_i - f_j 
\end{align*}$$ with all the expectations computed over $y$.

Each component $f_i$ (main effects) can be seen as the effect on $y$ of
varying $x_i$ alone. The same interpretation follows for $f_{ij}$ which
accounts for the (extra) variation of changing $x_i$ and $x_j$
simultaneously (second-order interaction). Higher-order terms have
analogous definitions.

The key step to decompose the variation of $y$ is to notice that $$
\text{var}(y) = E(y^2) - E(y)^2 = \int_0^1 f^2(\mathbf{ x}) \text{d}\mathbf{ x}- f_0^2
$$ and that this variance can be decomposed as $$ 
\text{var}(y) = \int_0^1 \sum_{i=1}^pf_i(x_i) \text{d}x_i + \int_0^1 \sum_{i<j}^{p} f_{ij}(x_i,x_j)\text{d} x_i \text{d} x_j + \cdots +\int_0^1 f_{1,2,\dots,d}(x_1,x_2,\dots,x_p)\text{d}\mathbf{ x}.
$$ This expression leads to the decomposition of the variance of $y$ as,
$$ 
\text{var}(y) = \sum_{i=1}^pV_i + \sum_{i<j}^{p} V_{ij} + \cdots + V_{12 \dots p},
$$ where $$ 
V_{i} = \text{var}_{x_i} \left( E_{\mathbf{ x}_{\sim i}} (y\mid x_{i}) \right),
$$ $$ 
V_{ij} = \text{var}_{x_{ij}} \left( E_{\mathbf{ x}_{\sim ij}} \left( y\mid x_i, x_j\right)\right) - \operatorname{V}_{i} - \operatorname{V}_{j}
$$ and so on. The $\mathbf{ x}_{\sim i}$ notation is used to indicate
all the set of variables but the $i^{th}$.

**Note**: The previous decomposition is important because it shows how
the variance in the output $y$ can be associated to each input or
interaction separately

Example: the Ishigami function
------------------------------

We illustrate the exact calculation of the Sobol indexes with the three
dimensional Ishigami function of (Ishigami and Homma, 1989). This is a
well-known example for uncertainty and sensitivity analysis methods
because of its strong nonlinearity and peculiar dependence on $x_3$.
More details of this function can be found in (Sobol and Levitan, 1999).

Mathematically, the from of the Ishigami function is $$
f(\textbf{x}) = \sin(x_1) + a \sin^2(x_2) + b x_3^4 \sin(x_1). 
$$ In this notebook we will set the parameters to be $a = 5$ and $b=0.1$
. The input variables are sampled randomly
$x_i \sim \text{Uniform}(-\pi,\pi)$.

Next we create the function object and visualize its shape marginally
for each one of its three inputs.

In [ ]:
from emukit.test_functions.sensitivity import Ishigami

In [ ]:
### --- Load the Ishigami function
ishigami = Ishigami(a=5, b=0.1)
target_simulator = ishigami.fidelity1

### --- Define the input space in which the simulator is defined
variable_domain = (-np.pi,np.pi)
x_grid = np.linspace(*variable_domain,100)
X, Y = np.meshgrid(x_grid, x_grid)

Before moving to any further analysis, we first plot the non-zero
components $f(\mathbf{ x})$. These components are $$\begin{align*}
f_1(x_1) & = \sin(x_1) \\
f_2(x_1) & = a \sin^2 (x_2) \\
f_{13}(x_1,x_3) & = b x_3^4 \sin(x_1) 
\end{align*}$$

In [ ]:
f1 = ishigami.f1(x_grid)
f2 = ishigami.f2(x_grid)
F13 = ishigami.f13(np.array([x_grid,x_grid]).T)[:,None]

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=plot.big_wide_figsize)
gs = axs[1, 1].get_gridspec()
# remove the underlying axes
for ax in axs[1, 0:]:
    ax.remove()

ax2 = fig.add_subplot(gs[1, 0:], projection='3d')

axs[0,0].plot(x_grid, f1,'-r')
axs[0,0].set_xlabel('$x_1$')
axs[0,0].set_ylabel('$f_1$')

axs[0,1].plot(x_grid,f2,'-r')
axs[0,1].set_xlabel('$x_2$')
axs[0,1].set_ylabel('$f_2$')

plt.suptitle('Non-zero Sobol components of the Ishigami function')

surf = ax2.plot_surface(X, Y, F13, cmap=cm.coolwarm, linewidth=0, antialiased=False)
ax2.set_xlabel('$x_1$')
ax2.set_ylabel('$x_3$')
ax2.set_zlabel('$f_{13}$')

mlai.write_figure(filename='non-zero-sobol-ishigami.svg', directory='./uq')

<img src="http://inverseprobability.com/talks/slides/../slides/diagrams/uq/non-zero-sobol-ishigami.svg" class="" width="80%" style="vertical-align:middle;">

Figure: <i>The non-zero components of the Ishigami function.</i>

The total variance $\text{var}(y)$ in this example is

In [ ]:
print(ishigami.variance_total)

which is the sum of the variance of $V_1$, $V_2$ and $V_{13}$

In [ ]:
print(ishigami.variance_x1, ishigami.variance_x2, ishigami.variance_x13)
print(ishigami.variance_x1 + ishigami.variance_x2 + ishigami.variance_x13)

First Order Sobol Indices using Monte Carlo
-------------------------------------------

The first order Sobol indexes are a measure of “first order sensitivity”
of each input variable. They account for the proportion of variance of
$y$ explained by changing each variable alone while marginalizing over
the rest. The Sobol index of the $i$th variable is computed as $$
S_i = \frac{V_i}{\text{var}(y)}.
$$ This value is standardized using the total variance so it is possible
to account for a fractional contribution of each variable to the total
variance of the output.

The Sobol indexes for higher order interactions $S_{ij}$ are computed
similarly. Note that the sum of all Sobol indexes equals to one.

In most cases we are interested in the first order indexes. In the
Ishigami function their values are:

In [ ]:
ishigami.main_effects

The most standard way of computing the Sobol indexes is using Monte
Carlo. Details are given in (Sobol, 2001).

With Emukit, the first-order Sobol indexes can be easily computed. We
first need to define the space where of target simulator is analyzed.

In [ ]:
from emukit.core import ContinuousParameter, ParameterSpace

In [ ]:
target_simulator = ishigami.fidelity1
variable_domain = (-np.pi,np.pi)

space = ParameterSpace(
          [ContinuousParameter('x1', variable_domain[0], variable_domain[1]), 
           ContinuousParameter('x2', variable_domain[0], variable_domain[1]),
           ContinuousParameter('x3', variable_domain[0], variable_domain[1])])

Compute the indexes is as easy as doing

In [ ]:
from emukit.sensitivity.monte_carlo import ModelFreeMonteCarloSensitivity

In [ ]:
np.random.seed(10)  # for reproducibility

num_mc = 10000  # Number of MC samples
senstivity_ishigami = ModelFreeMonteCarloSensitivity(target_simulator, space)
main_effects, total_effects, _ = senstivity_ishigami.compute_effects(num_monte_carlo_points = num_mc)
print(main_effects)

We compare the true effects with the Monte Carlo effects in a bar-plot.
The total effects are discussed later.

In [ ]:
import pandas as pd

In [ ]:
fig, ax = plt.subplots(figsize=plot.big_wide_figsize)

d = {'Sobol True': ishigami.main_effects,
     'Monte Carlo': main_effects}

pd.DataFrame(d).plot(kind='bar', ax=ax)
ax.set_title('First-order Sobol indices - Ishigami')
ax.set_ylabel('% of explained output variance')

mlai.write_figure(filename='first-order-sobol-indices-ishigami.svg', directory='./uq')

<img src="http://inverseprobability.com/talks/slides/../slides/diagrams/uq/first-order-sobol-indices-ishigami.svg" class="" width="80%" style="vertical-align:middle;">

Figure: <i>The non-zero components of the Ishigami function.</i>

Total Effects Using Monte Carlo
-------------------------------

Computing high order sensitivity indexes can be computationally very
demanding in high dimensional scenarios and measuring the total
influence of each variable on the variance of the output is infeasible.
To solve this issue the *total* indexes are used which account for the
contribution to the output variance of $x_i$ including all variance
caused by the variable alone and all its interactions of any order. The
total effect for $x_i$ is given by: $$ 
S_{Ti} = \frac{E_{\mathbf{ x}_{\sim i}} \left(\text{var}_{x_i} (y\mid \mathbf{ x}_{\sim i}) \right)}{\text{var}(y)} = 1 - \frac{\text{var}_{\mathbf{ x}_{\sim i}} \left(E_{x_i} (y\mid \mathbf{ x}_{\sim i}) \right)}{\text{var}(y)}
$$

Note that the sum of $S_{Ti}$ is not necessarily one in this case unless
the model is additive. In the Ishigami example the value of the total
effects is

In [ ]:
ishigami.total_effects

As in the previous example, the total effects can be computed with Monte
Carlo. In the next plot we show the comparison with the true total
effects.

In [ ]:
fig, ax = plt.subplots(figsize=plot.big_wide_figsize)

d = {'Sobol True': ishigami.total_effects,
     'Monte Carlo': total_effects}

pd.DataFrame(d).plot(kind='bar', ax=ax)
ax.set_title('Total effects - Ishigami')
ax.set_ylabel('Effects value')

mlai.write_figure(filename='total-effects-ishigami.svg', directory='./uq')

<img src="http://inverseprobability.com/talks/slides/../slides/diagrams/uq/total-effects-ishigami.svg" class="" width="80%" style="vertical-align:middle;">

Figure: <i>The total effects from the Ishigami function as computed via
Monte Carlo estimate alongside the true total effects for the Ishigami
function.</i>

Computing the sensitivity coefficients using the output of a model
------------------------------------------------------------------

In the example used above the Ishigami function is very cheap to
evaluate. However, in most real scenarios the functions of interest are
expensive and we need to limit ourselves to a few number of evaluations.
Using Monte Carlo methods is infeasible in these scenarios as a large
number of samples are typically required to provide good estimates of
the Sobol coefficients.

An alternative in these cases is to use Gaussaian process emulator of
the function of interest trained on a few inputs and outputs. If the
model is properly trained, its mean prediction which is cheap to
evaluate, can be used to compute the Monte Carlo estimates of the Sobol
coefficients. Let’s see how we can do this in Emukit.

We start by generating 100 samples in the input domain. Note that this a
just 1% of the number of samples that we used to compute the Sobol
coefficients using Monte Carlo.

In [ ]:
from emukit.core.initial_designs import RandomDesign

In [ ]:
desing = RandomDesign(space)
x = desing.get_samples(500)
y = ishigami.fidelity1(x)[:,None]

Now, we fit a standard Gaussian process to the samples and we wrap it as
an Emukit model.

In [ ]:
from GPy.models import GPRegression
from emukit.model_wrappers import GPyModelWrapper
from emukit.sensitivity.monte_carlo import MonteCarloSensitivity

In [ ]:
model_gpy = GPRegression(x,y)
model_emukit = GPyModelWrapper(model_gpy)
model_emukit.optimize()

The final step is to compute the coefficients using the class
`ModelBasedMonteCarloSensitivity` which directly calls the model and
uses its predictive mean to compute the Monte Carlo estimates of the
Sobol indices. We plot the true estimates, those computed using 10000
direct evaluations of the objecte using Monte Carlo and those computed
using a Gaussian process model trained on 100 evaluations.

In [ ]:
senstivity_ishigami_gpbased = MonteCarloSensitivity(model = model_emukit, input_domain = space)
main_effects_gp, total_effects_gp, _ = senstivity_ishigami_gpbased.compute_effects(num_monte_carlo_points = num_mc)

In [ ]:
fig, ax = plt.subplots(figsize=plot.big_wide_figsize)

main_effects_gp = {ivar: main_effects_gp[ivar][0] for ivar in main_effects_gp}

d = {'Sobol True': ishigami.main_effects,
     'Monte Carlo': main_effects,
     'GP Monte Carlo':main_effects_gp}

pd.DataFrame(d).plot(kind='bar', ax=ax)
plt.title('First-order Sobol indexes - Ishigami')
plt.ylabel('% of explained output variance')

mlai.write_figure(filename='first-order-sobol-indices-gp-ishigami.svg', directory='./uq')

<img src="http://inverseprobability.com/talks/slides/../slides/diagrams/uq/first-order-sobol-indices-gp-ishigami.svg" class="" width="80%" style="vertical-align:middle;">

Figure: <i>First Order sobol indices as estimated by Monte Carlo and
GP-emulator based Monte Carlo.</i>

In [ ]:
fig, ax = plt.subplots(figsize=plot.big_wide_figsize)

total_effects_gp = {ivar: total_effects_gp[ivar][0] for ivar in total_effects_gp}

d = {'Sobol True': ishigami.total_effects,
     'Monte Carlo': total_effects,
     'GP Monte Carlo':total_effects_gp}

pd.DataFrame(d).plot(kind='bar', ax=ax)
ax.set_title('Total effects - Ishigami')
ax.set_ylabel('% of explained output variance')

mlai.write_figure(filename='total-effects-sobol-indices-gp-ishigami.svg', directory='./uq')

<img src="http://inverseprobability.com/talks/slides/../slides/diagrams/uq/total-effects-sobol-indices-gp-ishigami.svg" class="" width="80%" style="vertical-align:middle;">

Figure: <i>Total effects as estimated by Monte Carlo and GP based Monte
Carlo.</i>

We observe some discrepacies with respect to the real value of the
coefficient when using the Gaussian process but we get a fairly good a
approximation a very reduced number of evaluations of the original
target function.

Conclusions
-----------

The Sobol indexes are a tool for explaining the variance of the output
of a function as components of the input variables. Monte Carlo is an
approach for computing these indexes if the function is cheap to
evaluate. Other approaches will be needed if $f(\cdot)$ is expensive to
compute.

Thanks!
-------

For more information on these subjects and more you might want to check
the following resources.

-   twitter: [@lawrennd](https://twitter.com/lawrennd)
-   podcast: [The Talking Machines](http://thetalkingmachines.com)
-   newspaper: [Guardian Profile
    Page](http://www.theguardian.com/profile/neil-lawrence)
-   blog:
    [http://inverseprobability.com](http://inverseprobability.com/blog.html)

References
----------

Ishigami, T., Homma, T., 1989. An importance quantification technique in
uncertainty analysis for computer models. \[1990\] Proceedings. First
International Symposium on Uncertainty Modeling and Analysis 398–403.

Kennedy, M.C., O’Hagan, A., 2000. Predicting the output from a complex
computer code when fast approximations are available. Biometrika 87,
1–13.

Saltelli, A., Annoni, P., Azzini, I., Campolongo, F., Ratto, M.,
Tarantola, S., 2010. Variance based sensitivity analysis of model
output. Design and estimator for the total sensitivity index. Computer
Physics Communications 181, 259–270.
<https://doi.org/10.1016/j.cpc.2009.09.018>

Saltelli, A., Ratto, M., Andres, T., Campolongo, F., Cariboni, J.,
Gatelli, D., Saisana, M., Tarantola, S., 2008. Global sensitivity
analysis: The primer. wiley.

Saltelli, A., Tarantola, S., Campolongo, F., Ratto, M., 2004.
Sensitivity analysis in practice: A guide to assessing scientific
methods. wiley.

Sobol, I.M., 2001. Global sensitivity indices for nonlinear mathematical
models and their Monte Carlo estimates. Mathematics and Computers in
Simulation 55, 271–280. <https://doi.org/10.1016/S0378-4754(00)00270-6>

Sobol, I.M., 1990. On sensitivity estimation for nonlinear mathematical
models. Matematicheskoe Modelirovanie 2, 112–118.

Sobol, I.M., Levitan, Y.L., 1999. On the use of variance reducing
multipliers in Monte Carlo computations of a global sensitivity index.
Computer Physics Communications 117, 52–61.
<https://doi.org/10.1016/S0010-4655(98)00156-8>